# 5 Minute strategy met Bollinger, QQE, MFI, Slowstoch

Indicators:

* Bollinger bands (length: 14, Std Deviation: 1)
* QQE (RSI lenght: 7, Smooth factor: 7, Factor: 2 (standaard 4.236))
* Money Flow Index (Length: 20)
* Slowstoch (SmoothK: 20, SmoothD: 10, Smooth: 10)

## Instapcondities

1. Candle close crossover boven de BB 1 dev
2. Instapcandle moet bullish zijn (close > open)
3. Instapcandle close moet hoger zijn dan die van 4 candles terug

Ten tijde van punten 1 en 2 moeten de onderstaande indicatoren bullish zijn:

1. Slowstoch bullish (K boven D)
2. QQE moet bullish zijn
3. MFI moet boven 50 zijn (en/of boven eigen gemiddelde van X - nog uit te zoeken)

## Uitstap condities

Als 0.4% winst is

## Stoploss

Candle sluit onder Bollinger Band Moving average


In [2]:
# Importeer de benodigde bibliotheken
import pandas as pd
import pandas_ta as pta
from pandas import DataFrame
import numpy as np

In [3]:
# Load data (LET OP: Ik gebruik hier 30min candles maar dit worden straks 5 min candles)
df = pd.read_json("./data/BTC_USDT-30m.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
67375,2021-06-26 05:30:00,32323.60,32357.09,32090.00,32229.32,969.118161
67376,2021-06-26 06:00:00,32229.32,32436.49,32172.62,32314.84,826.975210
67377,2021-06-26 06:30:00,32314.83,32349.01,31640.00,31731.25,2229.098192
67378,2021-06-26 07:00:00,31731.25,31820.01,31067.00,31227.31,6815.508898
67379,2021-06-26 07:30:00,31227.31,31290.62,30573.77,30720.37,5426.579597


In [4]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.2.45b0
Total Indicators: 133
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_doji, cdl_inside, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, decay, decreasing, dema, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, kama, kc, kdj, kst, kurtosis, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, ssf, stdev, stoch, stochrsi, supertrend, swma, t3, tema, thermo, trend_return, trima, trix, true_range, tsi, ttm_trend, ui, uo, variance, vidya, vortex, vp, vwap, vwma, wcp, willr, wma, zlm

## Bollinger bands

In dit voorbeeld wordt Pandas-ta Bollinger bands gebruikt. In de bot wordt de talib.abstract Bollinger gebruikt. 

In [5]:
help(pta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, mamode=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=20, std=2
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 20
        std (int): The long period. Default: 2
        mamode (str): Two options: "sma" or "ema". Default: "ema"
        offset (int): How many 

In [6]:
pta.bbands(close=df['close'], length=14, std=1, mamode=None, offset=None)

,BBL_14_1.0,BBM_14_1.0,BBU_14_1.0,BBB_14_1.0
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
67375,31701.091039,31949.526429,32197.961818,1.555174
67376,31777.963282,32007.730000,32237.496718,1.435695
67377,31806.620744,32018.215714,32229.810685,1.321716
67378,31675.624685,31973.401429,32271.178172,1.862653


In [7]:
df['bb_upper'] = pta.bbands(close=df['close'], length=14, std=1, mamode=None, offset=None)['BBU_14_1.0']
df['bb_middle'] = pta.bbands(close=df['close'], length=14, std=1, mamode=None, offset=None)['BBM_14_1.0']
df['bb_lower'] = pta.bbands(close=df['close'], length=14, std=1, mamode=None, offset=None)['BBL_14_1.0']

### QQE Signaal

QQE bepalen en QQE signal bepalen.

LET OP: qqe_rsi(ma) is de signal lijn en als deze boven of onder de qqe lijn komt, dan geeft deze bullish of bearish signalen.

* sig1 = qqe_sma > qqe 


In [8]:
help(pta.qqe)

Help on function qqe in module pandas_ta.momentum.qqe:

qqe(close, length=None, smooth=None, factor=None, mamode=None, drift=None, offset=None, **kwargs)
    Quantitative Qualitative Estimation (QQE)
    
    The Quantitative Qualitative Estimation (QQE) is similar to SuperTrend but uses a Smoothed RSI with an upper and lower bands. The band width is a combination of a one period True Range of the Smoothed RSI which is double smoothed using Wilder's smoothing length (2 * rsiLength - 1) and multiplied by the default factor of 4.236. A Long trend is determined when the Smoothed RSI crosses the previous upperband and a Short trend when the Smoothed RSI crosses the previous lowerband.
    
    Based on QQE.mq5 by EarnForex Copyright © 2010, based on version by Tim Hyder (2008), based on version by Roman Ignatov (2006)
    
    Sources:
        https://www.tradingview.com/script/IYfA9R2k-QQE-MT4/
        https://www.tradingpedia.com/forex-trading-indicators/quantitative-qualitative-estimati

In [9]:
pta.qqe(close=df['close'], length=7, smooth=7, factor=2, mamode=None, drift=None, offset=None)

,QQE_7_7_2.0,QQE_7_7_2.0_RSIMA,QQEl_7_7_2.0,QQEs_7_7_2.0
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
67375,53.901790,57.942047,53.901790,NaN
67376,55.252521,59.245825,55.252521,NaN
67377,49.444657,53.554278,49.444657,NaN
67378,42.166662,46.563219,42.166662,NaN


In [10]:
# QQE
df['qqe_rsi']=pta.qqe(close=df['close'], length=7, smooth=7, factor=2, mamode=None, drift=None, offset=None)['QQE_7_7_2.0_RSIMA']
df['qqe_atr']=pta.qqe(close=df['close'], length=7, smooth=7, factor=2, mamode=None, drift=None, offset=None)['QQE_7_7_2.0']
df.tail(25)

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,qqe_rsi,qqe_atr
67355,2021-06-25 19:30:00,32476.08,32478.79,32043.40,32220.74,1408.056785,32887.181026,32411.770714,31936.360403,34.883046,30.741670
67356,2021-06-25 20:00:00,32222.91,32288.91,31918.05,32288.90,1749.510905,32825.161073,32369.147857,31913.134641,37.486074,33.163812
67357,2021-06-25 20:30:00,32288.91,32369.00,32089.79,32199.99,1033.946116,32734.155779,32312.953571,31891.751363,38.715902,34.298758
67358,2021-06-25 21:00:00,32200.00,32200.00,31553.93,31813.30,2611.249506,32547.118640,32208.866429,31870.614217,37.045676,34.298758
67359,2021-06-25 21:30:00,31813.29,32228.11,31650.00,31799.37,1644.192399,32448.033998,32138.162143,31828.290288,35.711530,34.298758
67360,2021-06-25 22:00:00,31799.14,31975.98,31629.40,31844.29,1210.148178,32340.835234,32074.545000,31808.254766,35.340273,34.298758
67361,2021-06-25 22:30:00,31844.30,31885.84,31275.00,31423.32,2963.645529,32262.057200,31989.991429,31717.925658,32.603831,36.955543
67362,2021-06-25 23:00:00,31423.32,31628.57,31380.74,31499.99,1327.221092,32257.841628,31955.813571,31653.785514,31.638834,35.915797
67363,2021-06-25 23:30:00,31499.99,32000.00,31483.75,31584.45,1883.307409,32247.950089,31929.982143,31612.014197,32.141470,35.915797
67364,2021-06-26 00:00:00,31576.09,31918.77,31300.00,31854.71,3070.258762,32240.886229,31922.440714,31603.995200,35.909932,35.915797


## Money Flow Index

(Klinger VOlume Oscillator is gek genoeg niet aanwezig in Pandas_ta)

In [11]:
help(pta.mfi)

Help on function mfi in module pandas_ta.volume.mfi:

mfi(high, low, close, volume, length=None, drift=None, offset=None, **kwargs)
    Money Flow Index (MFI)
    
    Money Flow Index is an oscillator indicator that is used to measure buying and
    selling pressure by utilizing both price and volume.
    
    Sources:
        https://www.tradingview.com/wiki/Money_Flow_(MFI)
    
    Calculation:
        Default Inputs:
            length=14, drift=1
        tp = typical_price = hlc3 = (high + low + close) / 3
        rmf = raw_money_flow = tp * volume
    
        pmf = pos_money_flow = SUM(rmf, length) if tp.diff(drift) > 0 else 0
        nmf = neg_money_flow = SUM(rmf, length) if tp.diff(drift) < 0 else 0
    
        MFR = money_flow_ratio = pmf / nmf
        MFI = money_flow_index = 100 * pmf / (pmf + nmf)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        volume (pd.Series): 

In [12]:
pta.mfi(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], length=20, drift=None, offset=None)

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
67375    60.250447
67376    64.704349
67377    59.135394
67378    52.522437
67379    43.727254
Name: MFI_20, Length: 67380, dtype: float64

In [13]:
df['mfi'] = pta.mfi(high=df['high'], low=df['low'], close=df['close'], volume=df['volume'], length=20, drift=None, offset=None)
df.tail(25)

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,qqe_rsi,qqe_atr,mfi
67355,2021-06-25 19:30:00,32476.08,32478.79,32043.40,32220.74,1408.056785,32887.181026,32411.770714,31936.360403,34.883046,30.741670,36.721769
67356,2021-06-25 20:00:00,32222.91,32288.91,31918.05,32288.90,1749.510905,32825.161073,32369.147857,31913.134641,37.486074,33.163812,36.838254
67357,2021-06-25 20:30:00,32288.91,32369.00,32089.79,32199.99,1033.946116,32734.155779,32312.953571,31891.751363,38.715902,34.298758,39.026544
67358,2021-06-25 21:00:00,32200.00,32200.00,31553.93,31813.30,2611.249506,32547.118640,32208.866429,31870.614217,37.045676,34.298758,39.727845
67359,2021-06-25 21:30:00,31813.29,32228.11,31650.00,31799.37,1644.192399,32448.033998,32138.162143,31828.290288,35.711530,34.298758,44.035452
67360,2021-06-25 22:00:00,31799.14,31975.98,31629.40,31844.29,1210.148178,32340.835234,32074.545000,31808.254766,35.340273,34.298758,39.848010
67361,2021-06-25 22:30:00,31844.30,31885.84,31275.00,31423.32,2963.645529,32262.057200,31989.991429,31717.925658,32.603831,36.955543,40.675728
67362,2021-06-25 23:00:00,31423.32,31628.57,31380.74,31499.99,1327.221092,32257.841628,31955.813571,31653.785514,31.638834,35.915797,42.106322
67363,2021-06-25 23:30:00,31499.99,32000.00,31483.75,31584.45,1883.307409,32247.950089,31929.982143,31612.014197,32.141470,35.915797,41.638292
67364,2021-06-26 00:00:00,31576.09,31918.77,31300.00,31854.71,3070.258762,32240.886229,31922.440714,31603.995200,35.909932,35.915797,41.028619


## Slow stochastic

In [14]:
help(pta.stoch)

Help on function stoch in module pandas_ta.momentum.stoch:

stoch(high, low, close, k=None, d=None, smooth_k=None, offset=None, **kwargs)
    Stochastic (STOCH)
    
    The Stochastic Oscillator (STOCH) was developed by George Lane in the 1950's.
    He believed this indicator was a good way to measure momentum because changes in
    momentum precede changes in price.
    
    It is a range-bound oscillator with two lines moving between 0 and 100.
    The first line (%K) displays the current close in relation to the period's
    high/low range. The second line (%D) is a Simple Moving Average of the %K line.
    The most common choices are a 14 period %K and a 3 period SMA for %D.
    
    Sources:
        https://www.tradingview.com/wiki/Stochastic_(STOCH)
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=332&Name=KD_-_Slow
    
    Calculation:
        Default Inputs:
            k=14, d=3, smooth_k=3
        SMA = Simple Moving Average
        LL  = low 

In [15]:
pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=20, d=10, smooth_k=10, offset=None)

,STOCHk_20_10_10,STOCHd_20_10_10
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
67375,56.641217,37.136895
67376,60.935872,41.148630
67377,59.832597,44.920390
67378,56.678503,48.114069


In [16]:
df['stochk'] = pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=20, d=10, smooth_k=10, offset=None)['STOCHk_20_10_10']
df['stochd'] = pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=20, d=10, smooth_k=10, offset=None)['STOCHd_20_10_10']
df.tail(25)

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,qqe_rsi,qqe_atr,mfi,stochk,stochd
67355,2021-06-25 19:30:00,32476.08,32478.79,32043.40,32220.74,1408.056785,32887.181026,32411.770714,31936.360403,34.883046,30.741670,36.721769,16.530511,13.133314
67356,2021-06-25 20:00:00,32222.91,32288.91,31918.05,32288.90,1749.510905,32825.161073,32369.147857,31913.134641,37.486074,33.163812,36.838254,18.138879,13.753597
67357,2021-06-25 20:30:00,32288.91,32369.00,32089.79,32199.99,1033.946116,32734.155779,32312.953571,31891.751363,38.715902,34.298758,39.026544,19.934279,14.568363
67358,2021-06-25 21:00:00,32200.00,32200.00,31553.93,31813.30,2611.249506,32547.118640,32208.866429,31870.614217,37.045676,34.298758,39.727845,21.187691,15.509253
67359,2021-06-25 21:30:00,31813.29,32228.11,31650.00,31799.37,1644.192399,32448.033998,32138.162143,31828.290288,35.711530,34.298758,44.035452,21.494334,16.403215
67360,2021-06-25 22:00:00,31799.14,31975.98,31629.40,31844.29,1210.148178,32340.835234,32074.545000,31808.254766,35.340273,34.298758,39.848010,22.093559,17.344104
67361,2021-06-25 22:30:00,31844.30,31885.84,31275.00,31423.32,2963.645529,32262.057200,31989.991429,31717.925658,32.603831,36.955543,40.675728,22.167191,18.344542
67362,2021-06-25 23:00:00,31423.32,31628.57,31380.74,31499.99,1327.221092,32257.841628,31955.813571,31653.785514,31.638834,35.915797,42.106322,21.053184,19.113889
67363,2021-06-25 23:30:00,31499.99,32000.00,31483.75,31584.45,1883.307409,32247.950089,31929.982143,31612.014197,32.141470,35.915797,41.638292,20.503695,19.771776
67364,2021-06-26 00:00:00,31576.09,31918.77,31300.00,31854.71,3070.258762,32240.886229,31922.440714,31603.995200,35.909932,35.915797,41.028619,19.928577,20.303190


## Indicators gebruiken voor signalering

### Buy1 conditie

* StochK > StochD
* QQE_rsi > QQE_atr
* MFI > 50

In [32]:
df['buy1'] = (df['stochk'] > df['stochd']) & (df['qqe_rsi'] > df['qqe_atr']) & (df['mfi'] > 40)
df.tail(25)

,date,open,high,low,close,volume,bb_upper,bb_middle,bb_lower,qqe_rsi,qqe_atr,mfi,stochk,stochd,buy1,buy2,buy3,buy_final
67355,2021-06-25 19:30:00,32476.08,32478.79,32043.40,32220.74,1408.056785,32887.181026,32411.770714,31936.360403,34.883046,30.741670,36.721769,16.530511,13.133314,False,False,False,False
67356,2021-06-25 20:00:00,32222.91,32288.91,31918.05,32288.90,1749.510905,32825.161073,32369.147857,31913.134641,37.486074,33.163812,36.838254,18.138879,13.753597,False,False,False,False
67357,2021-06-25 20:30:00,32288.91,32369.00,32089.79,32199.99,1033.946116,32734.155779,32312.953571,31891.751363,38.715902,34.298758,39.026544,19.934279,14.568363,False,False,False,False
67358,2021-06-25 21:00:00,32200.00,32200.00,31553.93,31813.30,2611.249506,32547.118640,32208.866429,31870.614217,37.045676,34.298758,39.727845,21.187691,15.509253,False,False,False,False
67359,2021-06-25 21:30:00,31813.29,32228.11,31650.00,31799.37,1644.192399,32448.033998,32138.162143,31828.290288,35.711530,34.298758,44.035452,21.494334,16.403215,True,False,False,False
67360,2021-06-25 22:00:00,31799.14,31975.98,31629.40,31844.29,1210.148178,32340.835234,32074.545000,31808.254766,35.340273,34.298758,39.848010,22.093559,17.344104,False,False,False,False
67361,2021-06-25 22:30:00,31844.30,31885.84,31275.00,31423.32,2963.645529,32262.057200,31989.991429,31717.925658,32.603831,36.955543,40.675728,22.167191,18.344542,False,False,False,False
67362,2021-06-25 23:00:00,31423.32,31628.57,31380.74,31499.99,1327.221092,32257.841628,31955.813571,31653.785514,31.638834,35.915797,42.106322,21.053184,19.113889,False,False,False,False
67363,2021-06-25 23:30:00,31499.99,32000.00,31483.75,31584.45,1883.307409,32247.950089,31929.982143,31612.014197,32.141470,35.915797,41.638292,20.503695,19.771776,False,False,False,False
67364,2021-06-26 00:00:00,31576.09,31918.77,31300.00,31854.71,3070.258762,32240.886229,31922.440714,31603.995200,35.909932,35.915797,41.028619,19.928577,20.303190,False,False,False,False


### Buy2 conditie

* Close > BB upper band
* Close > Open
* CLose > Close.shift(4) (misschien een beetje onzin deze...)

In [33]:
# ALs candle hoger sluit dan BB upper
df['buy2'] = (df['close'] > df['bb_upper']) 
# Als er een verandering is van signaal en tegelijkertijd is close > open (green candle)
df['buy3'] =(df['buy2'].shift(+1) != df['buy2'])& (df['close'] > df['open'])
# Definitieve buy signaal als buy1 en buy3 True zijn
df['buy_final'] = (df['buy1'] == True) & (df['buy3'] == True)
# laat alle data zien waarbij alle condities tegelijkertijd waar zijn
df[['date','buy1','buy2','buy3','buy_final']][df['buy_final']==True].tail(25)

,date,buy1,buy2,buy3,buy_final
66566,2021-06-09 09:00:00,True,True,True,True
66569,2021-06-09 10:30:00,True,True,True,True
66653,2021-06-11 04:30:00,True,True,True,True
66662,2021-06-11 09:00:00,True,True,True,True
66664,2021-06-11 10:00:00,True,True,True,True
66732,2021-06-12 20:00:00,True,True,True,True
66757,2021-06-13 08:30:00,True,True,True,True
66766,2021-06-13 13:00:00,True,True,True,True
66871,2021-06-15 17:30:00,True,True,True,True
66893,2021-06-16 04:30:00,True,True,True,True
